# Case Study 2022 - Group 3

In [ ]:
from importlib import reload 
import augmentData
import loadAndStoreData
import processData
import drawImages
reload(augmentData)
reload(loadAndStoreData)
reload(processData)
reload(drawImages)

## Data Augmentation

In this section, the training data is augmented. The function allows to choose the classes for which the augmentations should be done.
It also allows to define the augmentation techniques that are used. 

For each augmentation technique a new subfolder is created. Each subfolder contains the augmented images of the classes chosen.
Depending on the augmentation techniques chosen, this process may neeed a minute or two.

In [ ]:
augmentData.performDataAugmentation(
    directory="training_patches/", 
    categories=["ponds", "pools","solar","trampoline"], 
    augmentations=["rotate_images", "move_images", "zoom_images", "change_brightness", "combine_augmentations"]
)


## Data Loading

This section loads the training patches into a numpy array and creates the corresponding label vector.
The result are X_train, X_val, y_train and y_val. 

The images are converted to RGB values, which is why there are 3 channels in the training data.

The training data sets are of dimension (number_of_instances x height x width x 3 channels). 
The label vectors only have one dimension (number_of_instances).

In [ ]:
training_data, labels = loadAndStoreData.loadTrainingDataAndLabels(
    folders=[
        "training_patches/",
        "training_patches_brightnessdown",
        "training_patches_brightnessup",
        "training_patches_combined",
        "training_patches_down",
        "training_patches_left",
        "training_patches_right",
        "training_patches_rotation",
        "training_patches_up",
        "training_patches_zoom"
    ], 
    subdirectories=["background", "ponds", "pools", "solar", "trampoline"])

In [ ]:
from sklearn.model_selection import train_test_split

labels_categorical = processData.labels_to_categorical(labels)
X_train, X_val, y_train, y_val = train_test_split(training_data, labels_categorical, test_size=0.33, random_state=1, stratify=labels)
print(X_train.shape)
print(y_train.shape)

## Model Training 

In [ ]:
y_train_encoded = processData.encodeLabels(y_train)

In [ ]:
from tensorflow.keras.layers import InputLayer, Dense, Flatten, Conv2D, MaxPool2D
from tensorflow import keras

model = keras.models.Sequential()
model.add(InputLayer(input_shape=(256,256,3)))
model.add(Conv2D(filters=10, kernel_size=(3,3), strides=1, padding="same", activation="relu"))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Flatten())
model.add(keras.layers.BatchNormalization())
model.add(keras.layers.Dense(256, activation='relu'))
model.add(keras.layers.Dropout(0.5))
model.add(keras.layers.BatchNormalization())
model.add(keras.layers.Dense(128, activation='relu'))
model.add(keras.layers.Dropout(0.5))
model.add(keras.layers.BatchNormalization())
model.add(keras.layers.Dense(64, activation='relu'))
model.add(keras.layers.Dropout(0.5))
model.add(keras.layers.BatchNormalization())
model.add(keras.layers.Dense(5, activation='softmax'))

model.summary()

In [ ]:
model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])


history = model.fit(X_train, 
                    y_train_encoded, 
                    epochs=20,
                    batch_size=64,
                    validation_split=0.1,
                   )

In [ ]:
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
import numpy as np

preds = model.predict(X_val)
preds_argmaxed = np.apply_along_axis(np.argmax, 1, preds)
f1_score(y_val,preds_argmaxed, average='macro'), accuracy_score(y_val, preds_argmaxed) 

In [ ]:
from sklearn.metrics import confusion_matrix

confusion_matrix(y_val, preds_argmaxed)

## Create predictions

In [ ]:
processData.makePredictions("validation_images", convnet=model, stepSize=64, windowSize=(256,256))

In [ ]:
import gc
preprocessed_patches = None
del preprocessed_patches
patch_coordinates = None
del patch_coordinates
X_train = None
del X_train
X_val = None 
del X_val
y_train = None
del y_train
y_val = None
training_data = None
del training_data
X_train_preprocessed = None
del X_train_preprocessed
predictions_array = None
del predictions_array
gc.collect()

In [ ]:
processData.nonMaxSuppressBoundingBoxes("validation_images/", iou_threshold=0.0, score_threshold=0.5)

## Draw Images with predictions

In [ ]:
drawImages.saveOrPrintImages(path="./validation_images", print_to_output=False, valBoundingBoxes=True,saveImagesPath="./validation_images", thickness=5)

## 